### Leitura dos Arquivos

In [107]:
import pandas as pd

def ler_arquivo(nome_df, path, col_codigo_empresa = 'Codigo Empresa', col_data_ref = 'Data Ref',col_valor_atual = 'Valor Trimestre Atual',col_cod_conta = 'Codigo Conta', 
                col_desc_conta = 'Descricao Conta'):
    df_transform = pd.read_csv(path, sep=";")
    df_transform['Codigo_Descricao_Conta'] = df_transform[col_cod_conta].astype(str) + ' - ' + df_transform[col_desc_conta]
    df_conta = df_transform[['Codigo Conta', 'Descricao Conta', 'Codigo_Descricao_Conta']].copy()
    df_conta['nome_df'] = nome_df
    df_transform.drop(columns=[col_cod_conta, col_desc_conta], inplace=True)
    df_transform = df_transform.pivot_table(index=[col_codigo_empresa, col_data_ref], columns='Codigo_Descricao_Conta', values= col_valor_atual, aggfunc='sum')
    df_transform = df_transform.rename_axis(columns=None)
    df_transform.reset_index(inplace=True)
    df_transform[col_data_ref] = pd.to_datetime(df_transform[col_data_ref], format='%Y%m%d')
    df_transform.sort_values(by=col_data_ref, inplace=True, ascending=False)
    
    return(df_transform, df_conta) # df transformado e um df_conta para ajudar a encontrar as contas sem precisar ir nos arquivos

df_cons_ativo, aux_conta_cons_ativo = ler_arquivo(nome_df = 'df_cons_ativo', path = f"Data\\02_Silver\\20240530_230717_DF Cons Ativo.csv")

df_cons_passivo, aux_conta_cons_passivo = ler_arquivo(nome_df = 'df_cons_passivo', path = f"Data\\02_Silver\\20240530_230717_DF Cons Passivo.csv")

df_cons_resultado_per, aux_conta_resultado_per = ler_arquivo(nome_df = 'df_cons_resultado_per', path = f"Data\\02_Silver\\20240530_230717_DF Cons Resultado Periodo.csv")

df_ind_passivo, aux_conta_ind_passivo = ler_arquivo(nome_df = 'df_ind_passivo', path = f"Data\\02_Silver\\20240530_230717_DF Ind Passivo.csv")

# Concatenar todos os DataFrames aux_conta
aux_conta = pd.concat([aux_conta_cons_ativo, aux_conta_cons_passivo, aux_conta_resultado_per, aux_conta_ind_passivo])

In [108]:
coluna = 'Descricao Conta'
texto = 'Empréstimos e Financiamentos'
aux_conta[aux_conta[coluna].str.contains(texto, case=False, na=False)]

,Codigo Conta,Descricao Conta,Codigo_Descricao_Conta,nome_df
14,2.01.04,Empréstimos e Financiamentos,2.01.04 - Empréstimos e Financiamentos,df_cons_passivo
15,2.01.04.01,Empréstimos e Financiamentos,2.01.04.01 - Empréstimos e Financiamentos,df_cons_passivo
45,2.02.01,Empréstimos e Financiamentos,2.02.01 - Empréstimos e Financiamentos,df_cons_passivo
46,2.02.01.01,Empréstimos e Financiamentos,2.02.01.01 - Empréstimos e Financiamentos,df_cons_passivo
125,2.01.04,Empréstimos e Financiamentos,2.01.04 - Empréstimos e Financiamentos,df_cons_passivo
126,2.01.04.01,Empréstimos e Financiamentos,2.01.04.01 - Empréstimos e Financiamentos,df_cons_passivo
156,2.02.01,Empréstimos e Financiamentos,2.02.01 - Empréstimos e Financiamentos,df_cons_passivo
157,2.02.01.01,Empréstimos e Financiamentos,2.02.01.01 - Empréstimos e Financiamentos,df_cons_passivo
237,2.01.04,Empréstimos e Financiamentos,2.01.04 - Empréstimos e Financiamentos,df_cons_passivo
238,2.01.04.01,Empréstimos e Financiamentos,2.01.04.01 - Empréstimos e Financiamentos,df_cons_passivo


### Criação dos indicadores

In [131]:
def merged_dfs(df1, df2, on, columns):
    merged_df = pd.merge(df1, df2, on=on)[columns]
    for col in columns:
        if merged_df[col].dtype == 'object':
            merged_df[col] = merged_df[col].str.replace('.', '').astype(float)
    return merged_df

def unique_dfs(df1, on, columns):
    unique_df = df1[columns].copy()
    for col in columns:
        if unique_df[col].dtype == 'object':
            unique_df[col] = unique_df[col].str.replace('.', '').astype(float)
    return unique_df

# Iniciando um df só com a data e código da empesa
df_indicadores = df_cons_ativo[["Codigo Empresa", "Data Ref"]]

# Liquidez_Corrente
indicador = ['Liquidez_Corrente']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['1.01 - Ativo Circulante', '2.01 - Passivo Circulante']

columns = on + columns_cal
merged_df = merged_dfs(df_cons_ativo, df_cons_passivo, on, columns)

merged_df[indicador[0]] = merged_df['1.01 - Ativo Circulante']/merged_df['2.01 - Passivo Circulante'] # Cálculo

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores


# Passivos_vs_Ativos"
indicador = ['Passivos_vs_Ativos']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.01 - Passivo Circulante', '2.02 - Passivo Não Circulante', '1 - Ativo Total']

columns = on + columns_cal
merged_df = merged_dfs(df_cons_ativo, df_cons_passivo, on, columns)

merged_df[indicador[0]] = (merged_df['2.01 - Passivo Circulante'] + merged_df['2.02 - Passivo Não Circulante'])/merged_df['1 - Ativo Total'] # Cálculo

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# PL
indicador = ['PL']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.03 - Patrimônio Líquido']

columns = on + columns_cal
merged_df = unique_dfs(df_ind_passivo, on, columns)

merged_df[indicador[0]] = merged_df['2.03 - Patrimônio Líquido']

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores


# PL_vs_Ativos
indicador = ['PL_vs_Ativos']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.03 - Patrimônio Líquido', '1 - Ativo Total']

columns = on + columns_cal
merged_df = merged_dfs(df_ind_passivo, df_cons_ativo, on, columns)
merged_df[indicador[0]] = merged_df['2.03 - Patrimônio Líquido']/merged_df['1 - Ativo Total']

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# Divida_Bruta
indicador = ['Divida_Bruta']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.01.04 - Empréstimos e Financiamentos', '2.02.01 - Empréstimos e Financiamentos']

columns = on + columns_cal
merged_df = unique_dfs(df_cons_passivo, on, columns)

merged_df[indicador[0]] = merged_df['2.01.04 - Empréstimos e Financiamentos'] + merged_df['2.02.01 - Empréstimos e Financiamentos']

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# Divida_Liquida
indicador = ['Divida_Liquida']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.01.04 - Empréstimos e Financiamentos', '2.02.01 - Empréstimos e Financiamentos', '1.01.01 - Caixa e Equivalentes de Caixa', '1.01.02 - Aplicações Financeiras']

columns = on + columns_cal
merged_df = merged_dfs(df_cons_passivo, df_cons_ativo, on, columns)

merged_df[indicador[0]] = (merged_df['2.01.04 - Empréstimos e Financiamentos'] + merged_df['2.02.01 - Empréstimos e Financiamentos']) - \
                            (merged_df['1.01.01 - Caixa e Equivalentes de Caixa'] + merged_df['1.01.02 - Aplicações Financeiras'])

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

,Codigo Empresa,Data Ref,Liquidez_Corrente,Passivos_vs_Ativos,PL,PL_vs_Ativos,Divida_Bruta,Divida_Liquida
0,7617,2024-03-31,2.706354,0.187061,80435000.0,0.773666,12189000.0,3803000.0
1,7617,2023-09-30,1.647388,0.184701,79738000.0,0.775548,11032000.0,2824000.0
2,7617,2023-06-30,1.494513,0.211235,76867000.0,0.751190,12606000.0,5000000.0
3,7617,2023-03-31,1.460624,0.209001,73632000.0,0.752307,12237000.0,5956000.0
4,7617,2021-09-30,1.592067,0.180318,62602000.0,0.772797,8186000.0,4257000.0
